<a href="https://colab.research.google.com/github/IrinaOltean14/Research/blob/main/PreprocessingDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-addons wandb pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.5/317.5 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.5 MB/s eta 0:00:00


In [4]:
pip install wandb

In [5]:
import wandb

In [7]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
import os

# Initialize wandb
wandb.init(project="semart-multi-task", name="ResNet18-multi-task-TF")

wandb: Currently logged in as: olteanirina1 (olteanirina1-babes-bolyai-university). Use `wandb login --relogin` to force relogin


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os

# Set the path to your Images folder
images_folder_path = '/content/drive/MyDrive/SemArt/Images'

# Count the number of files in the folder
image_count = len([f for f in os.listdir(images_folder_path) if f.endswith('.jpg')])

print(f"Total number of images in the folder: {image_count}")

Total number of images in the folder: 21424


In [11]:
import tensorflow as tf
import pandas as pd # Used for data cleaning, exploration, and preprocessing
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array # functions are used to load images from disk and convert them into NumPy arrays
from sklearn.preprocessing import LabelEncoder # categorical variables into numerical labels

In [14]:
# Define paths
images_folder_path = '/content/drive/MyDrive/SemArt/Images'
train_csv_path = '/content/drive/MyDrive/SemArt/semart_train.csv'
val_csv_path = '/content/drive/MyDrive/SemArt/semart_val.csv'
test_csv_path = '/content/drive/MyDrive/SemArt/semart_test.csv'

In [20]:
# Load csv. Prepare columns for image paths, school and date
def load_csv(csv_path):
    df = pd.read_csv(csv_path, sep='\t', encoding='ISO-8859-1')
    # Create full image path
    df['image_path'] = df['IMAGE_FILE'].apply(lambda x: os.path.join(images_folder_path, x))
    return df[['image_path', 'SCHOOL', 'TYPE']]

train_df = load_csv(train_csv_path)
val_df = load_csv(val_csv_path)
test_df = load_csv(test_csv_path)

In [22]:
# Vizualize data frames
# View the first 10 rows of the training DataFrame
print("Train Dataframe - Top 15:")
print(train_df.head(15))


Train Dataframe - Top 15:
                                           image_path         SCHOOL  \
0   /content/drive/MyDrive/SemArt/Images/19873-1da...         German   
1   /content/drive/MyDrive/SemArt/Images/18759-gua...        Italian   
2   /content/drive/MyDrive/SemArt/Images/04589-tem...        Italian   
3   /content/drive/MyDrive/SemArt/Images/15104-mag...  Netherlandish   
4   /content/drive/MyDrive/SemArt/Images/36582-pao...          Dutch   
5   /content/drive/MyDrive/SemArt/Images/00930-30c...        Italian   
6   /content/drive/MyDrive/SemArt/Images/24485-01v...        Italian   
7   /content/drive/MyDrive/SemArt/Images/18424-210...        Spanish   
8   /content/drive/MyDrive/SemArt/Images/21949-4la...        Italian   
9   /content/drive/MyDrive/SemArt/Images/03093-cal...        Flemish   
10  /content/drive/MyDrive/SemArt/Images/20320-mou...        Flemish   
11  /content/drive/MyDrive/SemArt/Images/08705-pie...         French   
12  /content/drive/MyDrive/SemArt/Imag

In [23]:
# We need to encode SCHOOL and TYPE into numerical labels
school_encoder = LabelEncoder()
type_encoder = LabelEncoder()

# Fit encoders on the train data (only train data to prevent data leakage)
train_df['SCHOOL'] = school_encoder.fit_transform(train_df['SCHOOL'])
train_df['TYPE'] = type_encoder.fit_transform(train_df['TYPE'])

# Apply encoders to validation and test splits
val_df['SCHOOL'] = school_encoder.transform(val_df['SCHOOL'])
val_df['TYPE'] = type_encoder.transform(val_df['TYPE'])
test_df['SCHOOL'] = school_encoder.transform(test_df['SCHOOL'])
test_df['TYPE'] = type_encoder.transform(test_df['TYPE'])


In [ ]:
IMG_SIZE = (224, 224)  # Image size for ResNet model

def process_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3) # output -> tensor representing the image
    img = tf.image.resize(img, IMG_SIZE) # ensures that all images have the same dimensions
    img = tf.keras.applications.resnet.preprocess_input(img) # specific preprocessing steps to the image, tailored for ResNet models
    return img

def process_labels(school, type_):
    school_label = tf.convert_to_tensor(school, dtype=tf.int32)
    type_label = tf.convert_to_tensor(type_, dtype=tf.int32)
    return school_label, type_label

